In [1]:
# Name = Anubhav Kumar Tiwary
# PRN: 2301208039

In [2]:
pip install pdfplumber PyPDF2 pdf2image pandas python-pptx beautifulsoup4 requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
!pip install ghostscript
import ghostscript


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import fitz  # PyMuPDF for PDF text extraction
import pdf2image
import camelot
import pandas as pd
from pptx import Presentation
from pptx.enum.shapes import MSO_SHAPE_TYPE
from bs4 import BeautifulSoup
import requests
import csv
import os

# Directory to store extracted images
IMAGE_DIR = "extracted_images"
os.makedirs(IMAGE_DIR, exist_ok=True)

def extract_from_pdf(pdf_path):
    pdf_data = {
        "textual content": "",
        "Images": [],
        "Table Content": [],
        "Links": []
    }
    
    # Open PDF file
    doc = fitz.open(pdf_path)
    for page_num in range(doc.page_count):
        page = doc[page_num]
        
        # Extract text and links
        pdf_data["textual content"] += page.get_text("text")
        pdf_data["Links"].extend([link['uri'] for link in page.get_links() if link['uri']])
        
        # Extract images
        for img_index, img in enumerate(page.get_images(full=True)):
            base_image = doc.extract_image(img[0])
            img_path = os.path.join(IMAGE_DIR, f"pdf_image_{page_num}_{img_index}.png")
            with open(img_path, "wb") as img_file:
                img_file.write(base_image["image"])
            pdf_data["Images"].append(img_path)

    # Extract tables
    tables = camelot.read_pdf(pdf_path, pages='all')
    for table in tables:
        pdf_data["Table Content"].append(table.df.to_dict())
    
    return pdf_data

def extract_from_pptx(pptx_path):
    ppt_data = {
        "textual content": "",
        "Images": [],
        "Table Content": [],
        "Links": []
    }
    
    # Load PowerPoint
    prs = Presentation(pptx_path)
    for slide in prs.slides:
        # Extract text
        for shape in slide.shapes:
            if shape.has_text_frame:
                ppt_data["textual content"] += shape.text + "\n"
            elif shape.shape_type == MSO_SHAPE_TYPE.PICTURE:
                img_path = os.path.join(IMAGE_DIR, f"ppt_image_{shape._element.get('id')}.png")
                with open(img_path, "wb") as img_file:
                    img_file.write(shape.image.blob)
                ppt_data["Images"].append(img_path)
        
        # Extract hyperlinks
        for shape in slide.shapes:
            if shape.has_text_frame and shape.click_action.hyperlink.address:
                ppt_data["Links"].append(shape.click_action.hyperlink.address)

        # Extract tables
        for shape in slide.shapes:
            if shape.has_table:
                table_data = []
                table = shape.table
                for row in table.rows:
                    row_data = [cell.text for cell in row.cells]
                    table_data.append(row_data)
                ppt_data["Table Content"].append(table_data)

    return ppt_data

def extract_from_url(url):
    url_data = {
        "textual content": "",
        "Images": [],
        "Table Content": [],
        "Links": []
    }
    
    # Fetch and parse the webpage
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Extract text
    url_data["textual content"] = soup.get_text()
    
    # Extract images
    for img in soup.find_all("img"):
        img_url = img.get("src")
        if img_url:
            url_data["Images"].append(img_url)
    
    # Extract links
    url_data["Links"] = [a.get("href") for a in soup.find_all("a", href=True)]
    
    # Extract tables
    for table in soup.find_all("table"):
        table_data = []
        for row in table.find_all("tr"):
            row_data = [cell.get_text(strip=True) for cell in row.find_all(["td", "th"])]
            table_data.append(row_data)
        url_data["Table Content"].append(table_data)

    return url_data

def save_to_csv(data, csv_filename):
    with open(csv_filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=["textual content", "Images", "Table Content", "Links"])
        writer.writeheader()
        writer.writerow({
            "textual content": data["textual content"],
            "Images": ", ".join(data["Images"]),
            "Table Content": str(data["Table Content"]),
            "Links": ", ".join(data["Links"])
        })

# Paths to the files
pdf_path = "Christopher Robert Evans.pdf"
pptx_path = "Skill_4_PPT.pptx"
url = "https://en.wikipedia.org/wiki/Elizabeth_Olsen"

# Extract and save data for each document type
pdf_data = extract_from_pdf(pdf_path)
save_to_csv(pdf_data, "pdf_data.csv")

ppt_data = extract_from_pptx(pptx_path)
save_to_csv(ppt_data, "ppt_data.csv")

url_data = extract_from_url(url)
save_to_csv(url_data, "url_data.csv")